# Import Packages

In [396]:
import pandas as pd
import numpy as np
import sklearn
import re
import math
import requests as re
import matplotlib.pyplot as plt
from operator import add
from keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
import seaborn as sns
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score, roc_auc_score, confusion_matrix, accuracy_score
from keras.models import Sequential
from keras import layers
from keras.utils import to_categorical, np_utils
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences

In [ ]:
Data = pd.read_csv("CleanData.csv").drop(columns = ["Unnamed: 0","LyricsOriginal"])
Data["TrackArtist"] = Data.Track +" By "+ Data.Artist1
Data.head()

# Artists Performance

In [ ]:
ArtistAll = Data.Artist2[Data.Artist2!="Nan"].value_counts()[0:10]
ArtistTop10 = Data.Artist2[(Data.Artist2!="Nan") & (Data.Rank <= 10)].value_counts()[0:10]
ArtistTop3 = Data.Artist2[(Data.Artist2!="Nan") & (Data.Rank <= 3)].value_counts()[0:10]
ArtistTop1 = Data.Artist2[(Data.Artist2!="Nan") & (Data.Rank <= 1)].value_counts()[0:10]
ArtistData = pd.DataFrame({
    "Rank" : list(range(1,11)),
    "MostWeeksWithSongsAtTopCharts" : [str(i)+" : "+str(j) for j, i in zip(ArtistTop1.tolist(),pd.DataFrame(ArtistTop1).index.tolist())],
    "MostWeeksWithSongsInTop3Charts" : [str(i)+" : "+str(j) for j, i in zip(ArtistTop3.tolist(),pd.DataFrame(ArtistTop3).index.tolist())],
    "MostWeeksWithSongsInTop10Charts" : [str(i)+" : "+str(j) for j, i in zip(ArtistTop10.tolist(),pd.DataFrame(ArtistTop10).index.tolist())],
    "MostWeeksWithSongsInTop100Charts" : [str(i)+" : "+str(j) for j, i in zip(ArtistAll.tolist(),pd.DataFrame(ArtistAll).index.tolist())]
})
ArtistData.to_csv("Leaderboards/ArtistLeaderboards.csv")
ArtistData.head()

# Songs Performance

In [ ]:
SongsAll = Data.TrackArtist[Data.TrackArtist!="Nan"].value_counts()[0:10]
SongsTop10 = Data.TrackArtist[(Data.TrackArtist!="Nan") & (Data.Rank <= 10)].value_counts()[0:10]
SongsTop3 = Data.TrackArtist[(Data.TrackArtist!="Nan") & (Data.Rank <= 3)].value_counts()[0:10]
SongsTop1 = Data.TrackArtist[(Data.TrackArtist!="Nan") & (Data.Rank <= 1)].value_counts()[0:10]
SongsData = pd.DataFrame({
    "Rank" : list(range(1,11)),
    "MostWeeksAtTopCharts" : [str(i)+" : "+str(j) for j, i in zip(SongsTop1.tolist(),pd.DataFrame(SongsTop1).index.tolist())],
    "MostWeeksInTop3Charts" : [str(i)+" : "+str(j) for j, i in zip(SongsTop3.tolist(),pd.DataFrame(SongsTop3).index.tolist())],
    "MostWeeksInTop10Charts" : [str(i)+" : "+str(j) for j, i in zip(SongsTop10.tolist(),pd.DataFrame(SongsTop10).index.tolist())],
    "MostWeeksInTop100Charts" : [str(i)+" : "+str(j) for j, i in zip(SongsAll.tolist(),pd.DataFrame(SongsAll).index.tolist())]
})
SongsData.to_csv("Leaderboards/SongLeaderboards.csv")
SongsData.head()

# Genre Data

In [ ]:
GenreData = pd.DataFrame({"Date" : Data.Date.unique()})
GenreData["Blues"] = [Data[(Data.Date == i) & (Data.Genre2=="Blues")].shape[0] for i in GenreData.Date]
GenreData["Rock"] = [Data[(Data.Date == i) & (Data.Genre2=="Rock")].shape[0] for i in GenreData.Date]
GenreData["Pop"] = [Data[(Data.Date == i) & (Data.Genre2=="Pop")].shape[0] for i in GenreData.Date]
GenreData["HipHop"] = [Data[(Data.Date == i) & (Data.Genre2=="HipHop")].shape[0] for i in GenreData.Date]
GenreData["R&B"] = [Data[(Data.Date == i) & (Data.Genre2=="R&B")].shape[0] for i in GenreData.Date]
GenreData["Country"] = [Data[(Data.Date == i) & (Data.Genre2=="Country")].shape[0] for i in GenreData.Date]
GenreData["Folk"] = [Data[(Data.Date == i) & (Data.Genre2=="Folk")].shape[0] for i in GenreData.Date]
GenreData["Reggae"] = [Data[(Data.Date == i) & (Data.Genre2=="Reggae")].shape[0] for i in GenreData.Date]
GenreData["Electro"] = [Data[(Data.Date == i) & (Data.Genre2=="Electro")].shape[0] for i in GenreData.Date]
GenreData["Religous"] = [Data[(Data.Date == i) & (Data.Genre2=="Religous")].shape[0] for i in GenreData.Date]
GenreData.head()

# Genre Output

In [ ]:
GenreData.to_csv("GenreData.csv")

# Build Unique Song Dataset For Neural Network

In [ ]:
UniqueData = Data[["TrackArtist","Artist2","Genre1","Genre2","LyricsClean"]].drop_duplicates()
UniqueData = UniqueData[UniqueData.TrackArtist.isna() == False]
UniqueData["MaxRank"] = [max(Data.Rank[Data.TrackArtist == i]) for i in UniqueData.TrackArtist]
UniqueData.head()

In [ ]:
plt.hist(UniqueData.MaxRank)

In [ ]:
for i in UniqueData.Genre2.unique():
    if pd.notna(i):
        fig, axs = plt.subplots(ncols=1, figsize=(15,5))
        sns.distplot(UniqueData.MaxRank[UniqueData.Genre2 == i], ax = axs).set_title("Highest Rank Achieved - {}".format(i))

# Split Training, Test, X & Y

In [ ]:
UniqueData = UniqueData.dropna(subset = ["LyricsClean"])

In [ ]:
UniqueData.to_csv("temp.csv")

In [711]:
UniqueData = pd.read_csv("temp.csv").drop(columns = "Unnamed: 0")
UniqueData.head()

,TrackArtist,Artist2,Genre1,Genre2,LyricsClean,MaxRank
0,Poor Little Fool By Ricky Nelson,Ricky Nelson,Pop/Rock,Rock,I used to play around with hearts that hastene...,71.0
1,Patricia By Perez Prado And His Orchestra,Perez Prado,Latin,NaN,Instrumental,85.0
2,Splish Splash By Bobby Darin,Bobby Darin,Pop/Rock,Blues,splish splash I was taking a bath long about a...,3.0
3,Hard Headed Woman By Elvis Presley With The Jo...,Elvis Presley,Pop/Rock,Rock,"Well a hard headed woman, A soft hearted man B...",94.0
4,Rebel-'rouser By Duane Eddy His Twangy Guitar ...,Duane Eddy,Pop/Rock,Rock,Instrumental,52.0


In [712]:
print("Numebr Of Nas For Genre1",len(UniqueData.Genre1)-UniqueData.Genre1.count())
UniqueData.Genre1.value_counts()

Numebr Of Nas For Genre1 343


Pop/Rock         10844
R&B               5073
Country           2184
Rap               1918
Vocal              829
Electronic         494
Jazz               227
Latin              198
Blues              180
Folk               171
Easy               144
Reggae             129
Stage               75
Religious           61
Comedy/Spoken       39
International       27
Children            18
Classical           12
New                  8
Avant                4
Holiday              1
Name: Genre1, dtype: int64

In [713]:
print("Numebr Of Nas For Genre2",len(UniqueData.Genre2)-UniqueData.Genre2.count())
UniqueData.Genre2.value_counts()

Numebr Of Nas For Genre2 4039


Rock        5239
Pop         4090
R&B         3011
Country     2221
HipHop      1871
Blues       1678
Folk         339
Electro      249
Reggae       168
Religous      74
Name: Genre2, dtype: int64

In [714]:
UniqueData = UniqueData.dropna(subset=["Genre1"])
UniqueData = UniqueData[UniqueData.Genre1.isin(["Pop/Rock"
                                               ,"R&B"       
                                               ,"Country"   
                                               ,"Rap"       
                                               ,"Vocal"     
                                               ,"Electronic"
                                               ,"Jazz"      
                                               ,"Latin"     
                                               ,"Blues"     
                                               ,"Folk"      
                                               ,"Easy"      
                                               ,"Reggae"])]
UniqueData.loc[UniqueData.Genre1=="Jazz"] = "Blues"
UniqueData.loc[UniqueData.Genre1=="Vocal"] = "Folk"
XTrain, XTest, YTrain, YTest = train_test_split(UniqueData["LyricsClean"], UniqueData['Genre1'])

# Bag Of Words

In [715]:
Vectorizer = TfidfVectorizer()
Vectorizer.fit(XTrain)
XTrain = Vectorizer.transform(XTrain)
XTest  = Vectorizer.transform(XTest)

# Baseline Logistic Regression Model

In [716]:
Classifier = LogisticRegression(n_jobs=-1)
Classifier.fit(XTrain, YTrain)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=-1,
          penalty='l2', random_state=None, solver='warn', tol=0.0001,
          verbose=0, warm_start=False)

In [717]:
UniqueData.Genre1.unique()

array(['Pop/Rock', 'Latin', 'R&B', 'Country', 'Blues', 'Folk', 'Easy',
       'Reggae', 'Electronic', 'Rap'], dtype=object)

In [718]:
print(f1_score(YTrain,Classifier.predict(XTrain), average="micro"))
print(f1_score(YTest,Classifier.predict(XTest), average="micro"))
confusion_matrix(YTest,Classifier.predict(XTest))

0.6823081045673792
0.6336191496963202


array([[  60,    0,    0,    0,    0,    0,   44,    6,    1,    0],
       [   0,   43,    0,    0,    0,    0,  431,   51,    4,    0],
       [   0,    0,    1,    0,    0,    0,   29,    5,    0,    0],
       [   0,    3,    0,    1,    0,    0,  104,   16,    4,    0],
       [   0,    0,    0,    0,  213,    0,   45,    9,    0,    0],
       [   0,    0,    0,    0,    0,    9,   35,    8,    0,    0],
       [   0,   10,    0,    0,    0,    3, 2500,  184,   14,    0],
       [   1,    5,    0,    0,    0,    0,  755,  443,   37,    0],
       [   0,    2,    0,    0,    0,    0,  185,   29,  273,    0],
       [   0,    0,    0,    0,    0,    0,   23,    4,    4,    4]])

# Baseline Deep Learning Model - Sequential Neural Network

In [719]:
Encoder = LabelEncoder()
Encoder.fit(YTrain)
YTrainNum = Encoder.transform(YTrain)
YTrainDummy = np_utils.to_categorical(YTrainNum)
YTestNum = Encoder.transform(YTest)
YTestDummy = np_utils.to_categorical(YTestNum)

In [722]:
InputDim = XTrain.shape[1]
Model = Sequential()
Model.add(layers.Dense(30, input_dim=InputDim, activation='relu'))
Model.add(layers.Dropout(rate = 0.4))
Model.add(layers.Dense(20, activation='relu'))
Model.add(layers.Dropout(rate = 0.4))
Model.add(layers.Dense(10, activation='softmax'))

Model.compile(loss='categorical_crossentropy', 
              optimizer="adam",
              metrics=["acc"])
Model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_239 (Dense)            (None, 30)                1131330   
_________________________________________________________________
dropout_50 (Dropout)         (None, 30)                0         
_________________________________________________________________
dense_240 (Dense)            (None, 20)                620       
_________________________________________________________________
dropout_51 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_241 (Dense)            (None, 10)                210       
Total params: 1,132,160
Trainable params: 1,132,160
Non-trainable params: 0
_________________________________________________________________


In [723]:
Model.fit(XTrain, YTrainDummy,
          epochs=20,
          validation_data=(XTest, YTestDummy),
          batch_size=25,
          verbose=1)

Train on 16793 samples, validate on 5598 samples
Epoch 1/20
16793/16793 [==============================] - 21s 1ms/step - loss: 1.5458 - acc: 0.4844 - val_loss: 1.2748 - val_acc: 0.5632
Epoch 2/20
16793/16793 [==============================] - 12s 724us/step - loss: 1.2428 - acc: 0.5710 - val_loss: 1.1472 - val_acc: 0.6068
Epoch 3/20
16793/16793 [==============================] - 12s 726us/step - loss: 1.0784 - acc: 0.6290 - val_loss: 1.0847 - val_acc: 0.6518
Epoch 4/20
16793/16793 [==============================] - 12s 720us/step - loss: 0.9462 - acc: 0.6789 - val_loss: 1.0667 - val_acc: 0.6606
Epoch 5/20
16793/16793 [==============================] - 12s 720us/step - loss: 0.8441 - acc: 0.7174 - val_loss: 1.0737 - val_acc: 0.6681
Epoch 6/20
16793/16793 [==============================] - 12s 724us/step - loss: 0.7628 - acc: 0.7461 - val_loss: 1.0818 - val_acc: 0.6740
Epoch 7/20
16793/16793 [==============================] - 12s 723us/step - loss: 0.7038 - acc: 0.7640 - val_loss: 1.117

In [725]:
print(f1_score(YTrainNum,Model.predict_classes(XTrain), average="micro"))
print(f1_score(YTestNum,Model.predict_classes(XTest), average="micro"))
confusion_matrix(YTestNum,Model.predict_classes(XTest))

0.905079497409635
0.6934619506966774


array([[  64,    3,    0,    4,    0,    0,   34,    6,    0,    0],
       [   1,  229,    0,    0,    2,    0,  217,   76,    4,    0],
       [   0,    0,    0,   11,    0,    0,   19,    5,    0,    0],
       [   1,    9,    0,   36,    0,    0,   56,   23,    3,    0],
       [   0,    3,    0,    0,  227,    0,   21,   15,    1,    0],
       [   1,    2,    0,    6,    0,   10,   14,   19,    0,    0],
       [   5,  107,    0,   18,    2,    5, 2162,  367,   45,    0],
       [   1,   34,    0,   10,    2,    0,  363,  800,   31,    0],
       [   1,    5,    0,    1,    2,    0,   88,   38,  354,    0],
       [   0,    1,    0,   10,    0,    0,   15,    6,    3,    0]])

# Word Embedding Neural Network

In [726]:
XTrain, XTest, YTrain, YTest = train_test_split(UniqueData["LyricsClean"], UniqueData['Genre1'])

In [727]:
Encoder = LabelEncoder()
Encoder.fit(YTrain)
YTrainNum = Encoder.transform(YTrain)
YTrainDummy = np_utils.to_categorical(YTrainNum)
YTestNum = Encoder.transform(YTest)
YTestDummy = np_utils.to_categorical(YTestNum)

In [728]:
TokenModel = Tokenizer(num_words=5000)
TokenModel.fit_on_texts(XTrain.tolist())
XTrain = TokenModel.texts_to_sequences(XTrain.tolist())
#XTrain = TokenModel.texts_to_matrix(XTrain, mode='tfidf')
XTest = TokenModel.texts_to_sequences(XTest.tolist())
#XTest = TokenModel.texts_to_matrix(XTest, mode='tfidf')

In [729]:
XTrain = pad_sequences(XTrain, padding='post', maxlen=500)
XTest = pad_sequences(XTest, padding='post', maxlen=500)

In [730]:
InputDim = XTrain.shape[1]
Model = Sequential()
Model.add(layers.Embedding(input_dim=5000,
                           output_dim=200,
                           input_length=500))
Model.add(layers.GlobalMaxPooling1D())
#Model.add(layers.Flatten())
Model.add(layers.Dense(50, activation='relu'))
#Model.add(layers.SpatialDropout1D(0.2))
#Model.add(layers.GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
#Model.add(layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2))
#Model.add(layers.Dense(40, activation='relu'))
#Model.add(layers.Dropout(rate = 0.3))
Model.add(layers.Dense(10, activation='softmax'))

Model.compile(loss='categorical_crossentropy', 
              optimizer="adam",
              metrics=["acc"])
Model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_73 (Embedding)     (None, 500, 200)          1000000   
_________________________________________________________________
global_max_pooling1d_24 (Glo (None, 200)               0         
_________________________________________________________________
dense_242 (Dense)            (None, 50)                10050     
_________________________________________________________________
dense_243 (Dense)            (None, 10)                510       
Total params: 1,010,560
Trainable params: 1,010,560
Non-trainable params: 0
_________________________________________________________________


In [731]:
Model.fit(XTrain, 
          YTrainDummy,
          epochs=20,
          validation_data=(XTest, YTestDummy),
          batch_size=25,
          verbose=1)

Train on 16793 samples, validate on 5598 samples
Epoch 1/20
16793/16793 [==============================] - 22s 1ms/step - loss: 1.3250 - acc: 0.5481 - val_loss: 1.1879 - val_acc: 0.5927
Epoch 2/20
16793/16793 [==============================] - 14s 834us/step - loss: 1.1117 - acc: 0.6258 - val_loss: 1.1190 - val_acc: 0.6286
Epoch 3/20
16793/16793 [==============================] - 14s 832us/step - loss: 0.9899 - acc: 0.6754 - val_loss: 1.0751 - val_acc: 0.6454
Epoch 4/20
16793/16793 [==============================] - 14s 832us/step - loss: 0.8639 - acc: 0.7230 - val_loss: 1.0719 - val_acc: 0.6651
Epoch 5/20
16793/16793 [==============================] - 14s 832us/step - loss: 0.7539 - acc: 0.7621 - val_loss: 1.0529 - val_acc: 0.6772
Epoch 6/20
16793/16793 [==============================] - 14s 832us/step - loss: 0.6506 - acc: 0.7949 - val_loss: 1.0791 - val_acc: 0.6720
Epoch 7/20
16793/16793 [==============================] - 14s 833us/step - loss: 0.5650 - acc: 0.8183 - val_loss: 1.099

In [734]:
print(f1_score(YTrainNum,Model.predict_classes(XTrain), average="micro"))
print(f1_score(YTestNum,Model.predict_classes(XTest), average="micro"))
confusion_matrix(YTestNum,Model.predict_classes(XTest))
print("😊")

0.9270529387244685
0.710075026795284
😊
